In [1]:
import torch
import torchvision
maskrcnn = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
fasterrcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)


In [9]:
from argparse import Namespace 
import torch
import numpy as np
from torch.utils.data import DataLoader
from src.detector.data_obj_detect import MOT16ObjDetect
from src.detector.utils import obj_detect_transforms

args = Namespace()
args.data_root_dir = "data/MOT16/train"
args.vis_threshold = 0.25
args.num_workers = 0
args.batch_size = 2

dataset_test = MOT16ObjDetect(
        root=args.data_root_dir,
        transforms=obj_detect_transforms(train=False),
        vis_threshold=args.vis_threshold,
        segmentation=True
    )

data_loader_test = DataLoader(
    dataset_test,
    batch_size=args.batch_size,
    shuffle=False,
    num_workers=args.num_workers,
    collate_fn=lambda batch: tuple(zip(*batch)),
)

In [16]:
images, targets = next(iter(data_loader_test))

In [18]:
targets[0].keys()

dict_keys(['boxes', 'labels', 'image_id', 'area', 'iscrowd', 'visibilities', 'masks'])

In [9]:
import matplotlib.pyplot as plt
import torchvision.ops as ops
from src.detector.utils import mask_convert

id = 1

boxes = targets[0]["boxes"][[id]]
boxes = ops.box_convert(boxes, "xyxy", "xywh")
instance_masks = targets[0]["masks"]
image = images[0]

#class_masks = (mask_convert(instance_masks, "scalar", "binary") != 0).float().sum(dim = 0)


fig, ax = plt.subplots()
ax.imshow(instance_masks[id])
for box in boxes:
    ax.add_patch(plt.Rectangle(box[:2], box[2], box[3], fill=False))

NameError: name 'targets' is not defined

In [77]:
from src.detector.utils import mask_convert
from src.utils.torch_utils import dict2keys_and_items
from src.tracker.data_track import MOT16Sequences



seqs = MOT16Sequences(root_dir="data/MOT16", dataset="MOT16-val", vis_threshold=0.0, segmentation=False)


In [3]:
import torch
path = "data/precomputed_detection/chair/preprocessed_data_train_2.pth"
db = torch.load(path)

In [ ]:

##################################################
# TOOOOOOOOOOOOOOO DOOOOOOOOOOOOOOOOOOOOOOOO
###################################################

########################################################
# in MOT16SequencePrecomputed eine option für reid einfügen
# if use_reid: -> frame["reid"] = reid -> return reid

In [ ]:
import torch.nn as nn

class M(nn.Module):
    def __init__(self):
        super().__init__()

In [13]:

# write existing database to my folder structure
import os
from src.utils.file_utils import ensure_dir
from src.tracker.data_track import MOT16Sequence
from tqdm import tqdm
dir = "data/precomputed_detection/default"

for seq_name in tqdm(db.keys(), desc="seqs"):
    dets = []
    for frame_id, frame in enumerate(db[seq_name], start=1):
        det = {"boxes": frame["det"]["boxes"], "scores": frame["det"]["scores"]}
        dets.append(det)
        reid_on_gt = frame["gt"]["reid"]
        reid_on_det = frame["det"]["reid"]
        reid_on_gt_path = os.path.join(dir, seq_name, "reid", "reid_on_gt", f"{frame_id:06d}.pth")
        reid_on_det_path = os.path.join(dir, seq_name, "reid", "reid_on_det", f"{frame_id:06d}.pth")
        ensure_dir(reid_on_gt_path)
        ensure_dir(reid_on_det_path)
        torch.save(reid_on_gt, reid_on_gt_path)
        torch.save(reid_on_det, reid_on_det_path)

    det_path = os.path.join(dir, seq_name, "detection.pth")
    ensure_dir(det_path)
    torch.save(dets, det_path)

        
    

seqs: 100%|██████████| 7/7 [00:06<00:00,  1.00it/s]
